In [13]:
from numpy.random import randint
from numpy.random import rand
import pandas as pd
import random as rd
from itertools import combinations
import math
#format of the data 
#[Loan Size(L), interest(rl),rating, loss(lambda)]
#Customers details provided in the question 
customers = [[10,0.021,'AAA',0.0002],
             [25,0.022,'BB',0.0058],
             [4,0.021,'A',0.001],
             [11,0.027,'AA',0.0003],
             [18,0.025,'BBB',0.0024],
             [3,0.026,'AAA',0.0002],
             [17,0.023,'BB',0.0058],
             [15,0.021,'AAA',0.0002],
             [9,0.028,'A',0.001],
             [10,0.022,'A',0.001]             
    
]
D = 60
K = 0.15
initial_sol=[1,0,0,1,0,1,0,1,0,1]

In [14]:
# objective function (Basically fitness function)
# rl * L  - lambda +  rt*Transactional cost  - 0.009*D - lambda
def fitness(x):
  sum  = -0.009*D
  for i in range(0,10):
    if x[i] == 1:
      sum = sum + customers[i][1]*customers[i][0] - 2*customers[i][3] + 0.01 * ((1-K)*D - customers[i][0]) 
  return (sum)

In [15]:
#Genetic Validation
def geneticvalidation(x):
  sum = 0
  for i in range(0,10):
    if x[i] == 1:
      sum = sum + customers[i][0] 
  return (sum)  

In [16]:
#GENERATING THE NEIGHBOURHOOD SOLUTION
def SwapMove(solution, i ,j):
    solution = solution.copy()
    solution[i], solution[j] = solution[j], solution[i]
    return solution



In [17]:
#PREPARING THR TABU STRUCTURE 
def get_tabuestructure():
    #dict = {{3,4}:{'ITERATIONS': 0,'FITNESS': 0}}
    dict = {}
    instance_dict = (0,1,2,3,4,5,6,7,8,9)
    for swap in combinations(instance_dict, 2):
        dict[swap] = {'ITERATIONS': 0, 'FITNESS': 0}
    return dict

In [18]:
def TabuSearch(initial_sol):
    tenure=4
    tabu_structure = get_tabuestructure() 
    best_solution=initial_sol
    best_objvalue=fitness(initial_sol)
    current_solution=initial_sol
    current_objvalue=fitness(initial_sol)
    iter = 1
    Terminate = 0
    while  Terminate < 10 and iter < 100:
        for move in tabu_structure:
            candidate_solution = SwapMove(current_solution, move[0], move[1])
            candidate_objvalue = fitness(candidate_solution)
            #validation
            if(geneticvalidation(candidate_solution)<= (1-K)*D):
              tabu_structure[move]['FITNESS'] = candidate_objvalue
            else:
              tabu_structure[move]['FITNESS'] = 0
        # Admissible move
        while True:
            # select the move with the Highest ObjValue in the neighborhood (minimization)
            best_move = max(tabu_structure, key =lambda x: tabu_structure[x]['FITNESS'])
            MoveValue = tabu_structure[best_move]["FITNESS"]
            tabu_time = tabu_structure[best_move]["ITERATIONS"]
            # Not Tabu
            if tabu_time < iter:
                # make the move
                current_solution = SwapMove(current_solution, best_move[0], best_move[1])
                current_objvalue = fitness(current_solution)
                # Best Improving move
                if MoveValue > best_objvalue:
                    best_solution = current_solution
                    best_objvalue = current_objvalue
                    Terminate = 0
                else:
                    Terminate += 1
                # update tabu_time for the move
                tabu_structure[best_move]['ITERATIONS'] = iter + tenure
                iter += 1
                break
            # If tabu
            else:
                # Aspiration
                if MoveValue > best_objvalue:
                    # make the move
                    current_solution = SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = fitness(current_solution)
                    best_solution = current_solution
                    best_objvalue = current_objvalue
                    Terminate = 0
                    iter += 1
                    break
                else:
                    tabu_structure[best_move]["FITNESS"] = 0
                    continue
    return best_solution

In [19]:
beste = TabuSearch(initial_sol)
print(beste, fitness(initial_sol))

[0, 0, 0, 1, 1, 1, 0, 0, 1, 1] 2.6362000000000005


In [20]:
validation(beste)

51